# Sequence To Sequence Model - 2 Syllable Words
---

## Imports

In [1]:
from matplotlib import pyplot as plt
import random
import pickle as pkl
import pandas as pd
import numpy as np
import math
from IPython.display import clear_output

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Conv1D, MaxPooling1D, MaxPooling2D, TimeDistributed, Input, Reshape, concatenate, Conv2D, Concatenate
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

---
## Load and Visualzie the Data

In [4]:
# load data in one line
ger_train = pkl.load(open("../../saved/ger_train_syls2.pkl", "rb"))
ger_test = pkl.load(open("../../saved/ger_test_syls2.pkl", "rb"))

In [5]:
# shape of the data
print(ger_train.shape)
print(ger_test.shape)

(3312, 2)
(2928, 2)


In [6]:
ger_train.head()

,contour,labels
0,"[0.03177828067373946, 0.03102610675462413, 0.0...",1
1,[],0
2,"[0.29295377456846683, 0.23372400800342857, 0.1...",0
3,"[0.3558331626873156, 0.39491268819019043, 0.41...",1
4,"[0.025010218876618886, 0.024972723808391342, 0...",1


---
## Padding

In [7]:
# calculate the max length of all contours
max_length_train = max([len(contour) for contour in ger_train["contour"]])
max_length_test = max([len(contour) for contour in ger_test["contour"]])
max_length = max(max_length_train, max_length_test)

In [8]:
# pad all contours to the same length using left padding, right padding, and both side padding
def pad_contours(contours, length, padding="both"):
    padded_contours = []
    for contour in contours:
        if padding == "left":
            padded_contours.append(np.pad(contour, (length-len(contour), 0), 'constant'))
        elif padding == "right":
            padded = np.pad(contour, (0, length-len(contour)), 'constant')
            padded_contours.append(padded)
        elif padding == "both":
            padded = np.pad(contour, (math.floor((length-len(contour))/2), math.ceil((length-len(contour))/2)), 'constant')
            padded_contours.append(padded)
    return np.array(padded_contours)

In [9]:
contours_train_rp = pad_contours(ger_train["contour"], max_length, padding="right")
contours_train_lp = pad_contours(ger_train["contour"], max_length, padding="left")
contours_train_bp = pad_contours(ger_train["contour"], max_length, padding="both")

contours_test_rp = pad_contours(ger_test["contour"], max_length, padding="right")
contours_test_lp = pad_contours(ger_test["contour"], max_length, padding="left")
contours_test_bp = pad_contours(ger_test["contour"], max_length, padding="both")

In [10]:
data_train = {
    'contours': contours_train_rp.tolist(),
    'labels': ger_train['labels'].tolist(),
}
data_test = {
    'contours': contours_test_rp.tolist(),
    'labels': ger_test['labels'].tolist(),
}

ger_train_padded = pd.DataFrame(data_train)
ger_test_padded = pd.DataFrame(data_test)

ger_test_padded.head()

,contours,labels
0,"[0.019162689527500977, 0.018757457994703516, 0...",1
1,"[0.018529939196573957, 0.01878130137324399, 0....",0
2,"[0.11961875005931769, 0.121682252123691, 0.127...",0
3,"[0.4515408450270049, 0.4999236321142739, 0.540...",1
4,"[0.01805156620491014, 0.016649114626452348, 0....",1


---
## Reformat
Reformat the dataset to make it compatible for sequencing

In [11]:
def reformat_dataset(dataset):
    input_sequences = []  # To store sequences of feature contours
    target_sequences = []  # To store sequences of labels

    current_word_contours = []  # To accumulate contours for the current word
    current_word_labels = []    # To accumulate labels for the current word

    for i in range(len(dataset)):
        contours, label = dataset[i]

        # Check if we're starting a new word
        if i % 2 == 0:
            current_word_contours.append(contours)
            current_word_labels.append(label)
        else:
            current_word_contours.append(contours)
            current_word_labels.append(label)

            # Append the sequences for the current word
            input_sequences.append(current_word_contours)
            target_sequences.append(current_word_labels)

            # Reset for the next word
            current_word_contours = []
            current_word_labels = []

    return np.array(input_sequences), np.array(target_sequences)

In [12]:
def dataframe_to_numpy(df):
    dataset = df[['contours', 'labels']].to_numpy()
    return dataset

In [13]:
ger_train_np = dataframe_to_numpy(ger_train_padded)
ger_test_np = dataframe_to_numpy(ger_test_padded)

In [14]:
input_seqs_train, target_seqs_train = reformat_dataset(ger_train_np)
input_seqs_test, target_seqs_test = reformat_dataset(ger_test_np)
type(input_seqs_train)

numpy.ndarray

In [15]:
print(input_seqs_train.shape)
print(target_seqs_train.shape)
print()
print(input_seqs_test.shape)
print(target_seqs_test.shape)

print(target_seqs_train)

(1656, 2, 138)
(1656, 2)

(1464, 2, 138)
(1464, 2)
[[1 0]
 [0 1]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]


# Seq 2 Seq Model

In [25]:
# Define input sequence length, syllable count, and feature contour size
input_seq_length = 138
syllable_count = 2
output_dim = 2  # Two classes: stressed and unstressed
latent_dim = 256  # Adjust the latent dimension as needed

In [26]:
# Define encoder inputs
encoder_inputs = Input(shape=(syllable_count, input_seq_length))

# Encoder LSTM
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# Discard encoder outputs and only keep the states
encoder_states = [state_h, state_c]

In [27]:
# Define decoder inputs
decoder_inputs = Input(shape=(None, output_dim))

# Decoder LSTM
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Dense layer for prediction
decoder_dense = Dense(output_dim, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [28]:
# Define the Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
# Print the model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 2, 138)]             0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, None, 2)]            0         []                            
                                                                                                  
 lstm_2 (LSTM)               [(None, 256),                404480    ['input_3[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                            

In [31]:
# One-hot encode the target labels
from keras.utils import to_categorical
num_classes = 2  # Stressed and unstressed
target_seqs_train_onehot = to_categorical(target_seqs_train, num_classes)
target_seqs_test_onehot = to_categorical(target_seqs_test, num_classes)
target_seqs_train_onehot[:2]

array([[[0., 1.],
        [1., 0.]],

       [[1., 0.],
        [0., 1.]]], dtype=float32)

In [32]:
# Train the model
model.fit(
    [input_seqs_train, target_seqs_train_onehot],  # Input and target data
    target_seqs_train_onehot,  # Labels for the decoder (teacher forcing)
    batch_size=64,
    epochs=30,
    validation_split=0.2  # Splitting a portion of the training data for validation
)

Epoch 1/30


21/21 [==============================] - 6s 82ms/step - loss: 0.6206 - accuracy: 0.6760 - val_loss: 0.4964 - val_accuracy: 0.7741
Epoch 2/30
21/21 [==============================] - 0s 21ms/step - loss: 0.3282 - accuracy: 0.9101 - val_loss: 0.1518 - val_accuracy: 0.9895
Epoch 3/30
21/21 [==============================] - 1s 46ms/step - loss: 0.1090 - accuracy: 0.9887 - val_loss: 0.0606 - val_accuracy: 0.9955
Epoch 4/30
21/21 [==============================] - 1s 35ms/step - loss: 0.0508 - accuracy: 0.9962 - val_loss: 0.0373 - val_accuracy: 1.0000
Epoch 5/30
21/21 [==============================] - 1s 27ms/step - loss: 0.0310 - accuracy: 0.9992 - val_loss: 0.0164 - val_accuracy: 1.0000
Epoch 6/30
21/21 [==============================] - 1s 29ms/step - loss: 0.0204 - accuracy: 0.9992 - val_loss: 0.0112 - val_accuracy: 1.0000
Epoch 7/30
21/21 [==============================] - 1s 42ms/step - loss: 0.0161 - accuracy: 0.9992 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 8/30


In [33]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate([input_seqs_test, target_seqs_test_onehot], target_seqs_test_onehot)

# Print the test results
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

46/46 [==============================] - 0s 9ms/step - loss: 0.0025 - accuracy: 0.9993
Test Loss: 0.0025205903220921755
Test Accuracy: 0.999316930770874


In [34]:
normal = target_seqs_test_onehot[0]
strange = target_seqs_test_onehot[1]

In [35]:
def one_hot_to_label(one_hot_encoded):
    # Convert one-hot encoded labels back to original labels
    return np.argmax(one_hot_encoded, axis=-1)


def visualize_prediction(model, encoder_input_data, decoder_input_data, target_data, index):
    # Get the input feature contours for the specified index
    encoder_input_seq = encoder_input_data[index]
    
    # Get the corresponding true labels in their original format
    true_labels = one_hot_to_label(target_data[index])
    
    # Reshape the input data to include batch dimension
    encoder_input_seq = np.expand_dims(encoder_input_seq, axis=0)
    decoder_input_seq = np.zeros((1, 2, num_classes))  # Initialize decoder input, assuming num_classes is defined
    
    # Predict using the model
    predicted_labels_one_hot = model.predict([encoder_input_seq, decoder_input_seq])
    
    # Convert predicted one-hot encoded labels back to original labels
    predicted_labels = one_hot_to_label(predicted_labels_one_hot)
    
    # Display the actual and predicted labels
    # print("Actual Labels:", true_labels)
    # print("Predicted Labels:", predicted_labels)
    x = all(predicted_labels[0][i] == true_labels[i] for i in range(len(true_labels)))
    # print("---------------")
    # print("(Correct)" if x else "(Wrong)")
    return x, true_labels, predicted_labels

In [36]:


print(normal)

normal_indices, strange_indices = [], []
for i, ele in enumerate(target_seqs_test_onehot):
    if np.array_equal(ele, normal): normal_indices.append(i)
    if np.array_equal(ele, strange): strange_indices.append(i)

print("len nomral = ", len(normal_indices))
print("len strange = ", len(strange_indices))

[[0. 1.]
 [1. 0.]]
len nomral =  1015
len strange =  446


In [37]:
target_seqs_test_onehot[1]

array([[1., 0.],
       [0., 1.]], dtype=float32)

## Tester Class
This class is designed to manually evaluate the performance of the model

In [38]:
class Tester:
    def __init__(self, model, A, B):
        self.model = model
        self.A = A
        self.B = B

    def check(self, i):
        x, t, p = visualize_prediction(model, input_seqs_test, np.zeros_like(input_seqs_test), target_seqs_test_onehot, index=i)
        print(t)
        print(p)
        print(x)

    def predict(self):
        self.A_correct = self.A_wrong = 0
        for a in self.A:
            x, _, _ = visualize_prediction(model, input_seqs_test, np.zeros_like(input_seqs_test), target_seqs_test_onehot, index=a)
            if x: self.A_correct += 1
            else: self.A_wrong += 1

        self.B_correct = self.B_wrong = 0
        for b in self.B:
            x, _, _ = visualize_prediction(model, input_seqs_test, np.zeros_like(input_seqs_test), target_seqs_test_onehot, index=b)
            if x: self.B_correct += 1
            else: self.B_wrong += 1

    def display(self):
        print("Class A")
        print("correct = ", self.A_correct)
        print("wrong = ", self.A_wrong)
        perc = (self.A_correct / (self.A_correct + self.A_wrong)) * 100
        print("Percentage = ", perc)
        
        print()

        print("Class B")
        print("correct = ", self.B_correct)
        print("wrong = ", self.B_wrong)
        perc = (self.B_correct / (self.B_correct + self.B_wrong)) * 100
        print("Percentage = ", perc)
        
        print()

        print("-------------------------")
        correct = self.A_correct + self.B_correct
        wrong = self.A_wrong + self.B_wrong
        print("correct = ", correct)
        print("wrong = ", wrong)
        perc = (correct / (correct + wrong)) * 100
        print("Percentage = ", perc)
        

In [39]:
tst = Tester(model, strange_indices, normal_indices)

In [40]:
tst.check(46)

1/1 [==============================] - 1s 1s/step
[1 0]
[[1 0]]
True


In [41]:
tst.predict()

1/1 [==============================] - 0s 29ms/step


In [42]:
tst.display()

Class A
correct =  119
wrong =  327
Percentage =  26.681614349775785

Class B
correct =  439
wrong =  576
Percentage =  43.251231527093594

-------------------------
correct =  558
wrong =  903
Percentage =  38.19301848049281
